<a href="https://colab.research.google.com/github/remcovanwijk040/AI/blob/main/kaggle/titanic_competition/script_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [16]:
# Om te importen moet je een terminal openen (vs code) en de volende code invullen: py -m pip install numpy
import pandas as pd
import numpy as np
import matplotlib as plt
import re

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score

pd.set_option("display.max.columns", None) #Make sure all columns are showed


## Import data

In [2]:
df_train = pd.read_csv("https://raw.githubusercontent.com/remcovanwijk040/AI/main/kaggle/titanic_competition/train.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/remcovanwijk040/AI/main/kaggle/titanic_competition/test.csv")
df_train.columns = df_train.columns.str.lower()
df_test.columns = df_test.columns.str.lower()

print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  891 non-null    int64  
 1   survived     891 non-null    int64  
 2   pclass       891 non-null    int64  
 3   name         891 non-null    object 
 4   sex          891 non-null    object 
 5   age          714 non-null    float64
 6   sibsp        891 non-null    int64  
 7   parch        891 non-null    int64  
 8   ticket       891 non-null    object 
 9   fare         891 non-null    float64
 10  cabin        204 non-null    object 
 11  embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  418 non-null    int64  
 1   pcl

## Check Missings

In [3]:
print(df_train.isnull().sum())
print(df_test.isnull().sum())


passengerid      0
survived         0
pclass           0
name             0
sex              0
age            177
sibsp            0
parch            0
ticket           0
fare             0
cabin          687
embarked         2
dtype: int64
passengerid      0
pclass           0
name             0
sex              0
age             86
sibsp            0
parch            0
ticket           0
fare             1
cabin          327
embarked         0
dtype: int64


Cabin en Age hebben veel missings -> Hier moeten we iets mee

Missings bij embarked en fare verwijderen we voor het gemak.

In [13]:
df_train = df_train.dropna(subset=['embarked'])
df_test = df_test.dropna(subset=['fare'])

print(df_train.isnull().sum())
print(df_test.isnull().sum())

passengerid      0
survived         0
pclass           0
name             0
sex              0
age            177
sibsp            0
parch            0
ticket           0
fare             0
cabin          687
embarked         0
age_group        0
title            0
cabin_floor      0
dtype: int64
passengerid      0
pclass           0
name             0
sex              0
age             86
sibsp            0
parch            0
ticket           0
fare             0
cabin          326
embarked         0
dtype: int64


In [5]:
# Missing data oplossen
pd.crosstab(index=df_train['survived'].fillna('missing'), columns=df_train['age'].fillna('missing'),
 dropna=False, normalize=False, margins = True, margins_name= "Total")

pd.crosstab(index=df_train['survived'].fillna('missing'), columns=df_train['cabin'].fillna('missing'),
 dropna=False, normalize=False, margins = True, margins_name= "Total")

cabin,A10,A14,A16,A19,A20,A23,A24,A26,A31,A32,A34,A36,A5,A6,A7,B101,B102,B18,B19,B20,B22,B3,B30,B35,B37,B38,B39,B4,B41,B42,B49,B5,B50,B51 B53 B55,B57 B59 B63 B66,B58 B60,B69,B71,B73,B77,B78,B79,B80,B82 B84,B86,B94,B96 B98,C101,C103,C104,C106,C110,C111,C118,C123,C124,C125,C126,C128,C148,C2,C22 C26,C23 C25 C27,C30,C32,C45,C46,C47,C49,C50,C52,C54,C62 C64,C65,C68,C7,C70,C78,C82,C83,C85,C86,C87,C90,C91,C92,C93,C95,C99,D,D10 D12,D11,D15,D17,D19,D20,D21,D26,D28,D30,D33,D35,D36,D37,D45,D46,D47,D48,D49,D50,D56,D6,D7,D9,E10,E101,E12,E121,E17,E24,E25,E31,E33,E34,E36,E38,E40,E44,E46,E49,E50,E58,E63,E67,E68,E77,E8,F E69,F G63,F G73,F2,F33,F38,F4,G6,T,missing,Total
survived,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,1,0,1,0,0,1,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,1,1,2,0,0,1,0,1,2,2,1,0,0,1,0,1,0,0,0,0,1,1,0,0,1,1,1,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,1,1,1,0,1,0,0,1,2,1,0,1,0,2,1,481,549
1,0,0,1,0,1,1,0,1,1,0,1,0,0,1,0,1,0,2,0,2,1,1,0,2,0,0,1,1,1,1,2,2,1,1,2,1,1,0,1,2,1,1,1,0,0,0,4,1,1,1,1,0,0,0,1,0,2,2,0,1,1,1,2,0,1,1,0,1,0,1,2,1,1,1,1,1,1,1,0,1,1,0,0,1,0,2,2,0,1,2,1,1,1,2,1,2,1,0,1,0,2,2,2,1,1,0,1,0,1,0,1,0,1,1,1,3,1,2,1,2,2,0,2,1,1,0,1,1,0,1,1,0,0,1,1,0,2,1,0,0,2,3,0,2,2,0,206,340
Total,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,2,2,1,1,2,1,1,1,1,1,1,2,2,1,2,2,2,1,1,1,2,1,1,1,1,1,1,4,1,1,1,1,1,1,1,2,2,2,2,1,1,2,3,4,1,1,1,1,1,1,1,2,1,1,2,2,1,1,2,1,2,1,1,1,1,1,2,2,1,1,3,1,1,1,2,1,2,1,2,1,1,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,3,1,2,1,2,2,1,2,1,1,1,1,2,1,1,1,1,1,2,1,1,2,1,1,2,3,3,1,2,4,1,687,889


In [6]:
# Cabin verder doorkijken
df_train[["cabin", "passengerid"]]
df_train.head()
## Je kunt dus achterhalen dat personen in één kamer verbleven. 
## Dat is interessant om te achterhalen wie de kinderen zijn van wie. De 
## voorspelling is namelijk dat moeder met kinderen de meeste kans hebben
## om te overleven en vaders met kinderen juist de minste.

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Restructure data

In [7]:
print(df_train.apply(lambda col: col.unique()))

# Verschillende kollomen bevatten nan. Dit wordt niet gelezen als missing dus passen we die aan naar NaN
df_train = df_train.replace("nan", "NaN")


passengerid    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
survived                                                  [0, 1]
pclass                                                 [3, 1, 2]
name           [Braund, Mr. Owen Harris, Cumings, Mrs. John B...
sex                                               [male, female]
age            [22.0, 38.0, 26.0, 35.0, nan, 54.0, 2.0, 27.0,...
sibsp                                      [1, 0, 3, 4, 2, 5, 8]
parch                                      [0, 1, 2, 5, 3, 4, 6]
ticket         [A/5 21171, PC 17599, STON/O2. 3101282, 113803...
fare           [7.25, 71.2833, 7.925, 53.1, 8.05, 8.4583, 51....
cabin          [nan, C85, C123, E46, G6, C103, D56, A6, C23 C...
embarked                                               [S, C, Q]
dtype: object


In [8]:
# Groeperen:
## Age -> agegroups
df_train.age.describe()

bins = [0, 9, 18, 24 ,30, 40, 50, 70, 100]
group_names = ["0-9", "10-18","19-24", "25-30","31-40", "41-50", "51-70", "70+"]
df_train['age_group'] = pd.cut(df_train['age'], bins, labels=group_names)

df_train.age_group.value_counts(dropna= False)

NaN      177
31-40    154
19-24    138
25-30    132
41-50     86
10-18     77
0-9       62
51-70     58
70+        5
Name: age_group, dtype: int64

In [9]:
# Check kinderen onder 18 en relatie tot ouders
temp = df_train[df_train['age'] <= 18]

pd.crosstab(index=temp['parch'], columns=temp['age'] , dropna=False, normalize=False, margins = True, margins_name= "Total")


age,0.42,0.67,0.75,0.83,0.92,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,14.5,15.0,16.0,17.0,18.0,Total
parch,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,3,1,3,11,10,18,50
1,1,1,2,1,0,3,7,4,6,1,2,2,3,2,0,0,0,1,1,0,2,4,1,5,49
2,0,0,0,1,1,4,3,2,4,2,1,1,1,6,2,3,0,0,2,0,0,1,2,3,39
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
Total,1,1,2,2,1,7,10,6,10,4,3,3,4,8,2,4,1,2,6,1,5,17,13,26,139


In [10]:
# Kolommen bewerken
## Name naar title
### Exract titel uit naam:
df_train["title"] = df_train["name"].str.extract('((?<=,)\s[a-zA-Z]*)')

## Cabin nummer
df_train["cabin_floor"] = df_train["cabin"].str.extract('([A-Z])')

print(df_train.title.value_counts(dropna = False))
print(df_train.cabin_floor.value_counts(dropna = False))

 Mr          517
 Miss        181
 Mrs         124
 Master       40
 Dr            7
 Rev           6
 Mlle          2
 Major         2
 Col           2
 the           1
 Capt          1
 Ms            1
 Sir           1
 Lady          1
 Mme           1
 Don           1
 Jonkheer      1
Name: title, dtype: int64
NaN    687
C       59
B       45
D       33
E       32
A       15
F       13
G        4
T        1
Name: cabin_floor, dtype: int64


## Missings opvullen

In [11]:
# Voor nu vul ik alle missings op met een categorie onbekend:
df_train["age_group"] = df_train["age_group"].cat.add_categories(['Onbekend'])
df_train["age_group"] = df_train["age_group"].fillna("Onbekend")

df_train["cabin_floor"] = df_train["cabin_floor"].fillna("Onbekend")

print(df_train.age_group.value_counts(dropna = False))
print(df_train.cabin_floor.value_counts(dropna = False))

Onbekend    177
31-40       154
19-24       138
25-30       132
41-50        86
10-18        77
0-9          62
51-70        58
70+           5
Name: age_group, dtype: int64
Onbekend    687
C            59
B            45
D            33
E            32
A            15
F            13
G             4
T             1
Name: cabin_floor, dtype: int64


## Preprocessing dataset

### Preparing features

### Spliting training data

In [26]:
# Split the training dataset in training en testing
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in split.split(df_train, df_train['survived']):
  strat_train_set = df_train.iloc[train_index]
  strat_test_set = df_train.iloc[test_index]

#check if this resulted in a 70% train set and 30% test set with equal outcome distributions
df = df_train
y = "survived"
print('share of observations in the train set:', str(round(len(strat_train_set)/len(df), 3)))
print('share of observations in the test set:', str(round(len(strat_test_set)/len(df), 3)))
print('share of 1\'s in the train set:', str(round(len(strat_train_set.loc[strat_train_set[y] == 1])/len(strat_train_set), 3)))
print('share of 1\'s in the test set:', str(round(len(strat_test_set.loc[strat_test_set[y] == 1])/len(strat_test_set), 3)))

share of observations in the train set: 0.7
share of observations in the test set: 0.3
share of 1's in the train set: 0.383
share of 1's in the test set: 0.382


In [ ]:
# split features and outcome data
X_train = strat_train_set.loc[:,strat_train_set.columns != 'Class']
y_train = strat_train_set['Class']
X_test = strat_test_set.loc[:,strat_test_set.columns != 'Class']
y_test = strat_test_set['Class']

### Setting up the pipeline

    selecting the variables
    imputing missing values
    scaling numeric variables
    creating dummy variables for categorical variables


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
        ('cat_imputer', SimpleImputer(strategy = 'most_frequent')),
        ('cat_encoder', OneHotEncoder(sparse=False))])

from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_features),
        ("cat", cat_pipeline, cat_features)])

num_features = X_train.select_dtypes(include=['float']).columns
cat_features = X_train.select_dtypes(exclude=['float']).columns
X_train_prep = full_pipeline.fit_transform(X_train)

## Train the dataset

In [146]:
# Categoriseren
## python machine learning kan alleen overweg met numerieke data, daarom doe ik
## alles omvormen met de onderstaande functie
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
cat_list =  ["sex", "pclass", "embarked", "title", "cabin_floor", "age_group"]

for x in cat_list:
  df_train[x + '_cat'] = le.fit_transform(df_train[x])
  print(list(le.classes_))

  ## @Eigenlijk moet ik hier de classes omdraaien, 1 betekent namelijk eerste klas
  ## het idee is dat zij "meer" zijn dan 3e klas in het kader van overlevingskans
  

['female', 'male']
[1, 2, 3]
['C', 'Q', 'S']
[' Capt', ' Col', ' Don', ' Dr', ' Jonkheer', ' Lady', ' Major', ' Master', ' Miss', ' Mlle', ' Mme', ' Mr', ' Mrs', ' Ms', ' Rev', ' Sir', ' the']
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'Onbekend', 'T']
['0-9', '10-18', '19-24', '25-30', '31-40', '41-50', '51-70', '70+', 'Onbekend']


In [149]:
## Set y and X
features = ["pclass_cat", "sex_cat", "age_group_cat", "cabin_floor_cat", "title_cat", "embarked_cat", "fare", "parch", "sibsp"]
y = df_train["survived"]
x = df_train[features]

x

,pclass_cat,sex_cat,age_group_cat,cabin_floor_cat,title_cat,embarked_cat,fare,parch,sibsp
0,2,1,2,7,11,2,7.2500,0,1
1,0,0,4,2,12,0,71.2833,0,1
2,2,0,3,7,8,2,7.9250,0,0
3,0,0,4,2,12,2,53.1000,0,1
4,2,1,4,7,11,2,8.0500,0,0
...,...,...,...,...,...,...,...,...,...
886,1,1,3,7,14,2,13.0000,0,0
887,0,0,2,1,8,2,30.0000,0,0
888,2,0,8,7,8,2,23.4500,2,1
889,0,1,3,2,11,0,30.0000,0,0
